This script takes the basic neural network model to estimate tempearture developed in `NeuralNetworks/_basic_leaky_model.py` and applies it to weather data and static reservoir elements to estimate reservoir temeperature for Granby, Grand, and Shadow Mountain.

In [14]:
#high level modules
import os
import sys
import imp
import numpy as np
import pandas as pd
import pickle

# ml/ai modules
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# import pydot
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors

# custom modules
this_dir = "/Users/steeleb/Documents/GitHub/ATS-ML-Fall2023/"
imp.load_source("universals", os.path.join(this_dir, "universal_functions.py"))
from universals import load_pickle_file, predict_values_test

First, we'll load the LOO models that we saved from the `NeuralNetworks/_basic_leaky_model.py` script.

In [26]:
# model directory path
model_dir = '/Users/steeleb/OneDrive - Colostate/NASA-Northern/data/NN_train_val_test/models/leaky_basic/'

models = [f for f in os.listdir(model_dir) if 'model' in f]

models = [f for f in models if 'ts' not in f]
models.sort()

model_1 = load_pickle_file(models[0], model_dir)
model_2 = load_pickle_file(models[1], model_dir)
model_3 = load_pickle_file(models[2], model_dir)
model_4 = load_pickle_file(models[3], model_dir)
model_5 = load_pickle_file(models[4], model_dir)
model_6 = load_pickle_file(models[5], model_dir)

And now we'll read in the weather data and reservoir elements.

In [29]:
app_dir = "~/OneDrive - Colostate/NASA-Northern/data/NN_train_val_test/application/"
standardized_df = pd.read_csv(os.path.join(app_dir, "application_dataset_2020-2022_apr-nov_standardized.csv"))
#standardized_df = standardized_df.set_index("index")
standardized_df


,elevation,area,shoreline_length,max_depth,volume,max_temp_degC_1,mean_temp_degC_1,min_temp_degC_1,tot_sol_rad_Wpm2_1,max_temp_degC_3,...,tot_precip_mm_5,min_wind_mps_1,mean_wind_mps_1,max_wind_mps_1,min_wind_mps_3,mean_wind_mps_3,max_wind_mps_3,min_wind_mps_5,mean_wind_mps_5,max_wind_mps_5
0,-0.717147,-0.441298,-0.448934,-0.090037,-0.240685,-0.809698,-0.952902,-0.965521,-0.129181,-1.085391,...,0.283029,1.153144,0.425586,-0.075665,0.451346,0.075579,-0.197092,0.251670,0.348472,0.246117
1,-0.717147,-0.441298,-0.448934,-0.090037,-0.240685,-0.484418,-0.375192,-0.313112,-0.300228,-0.740732,...,0.086421,-1.202816,-0.440772,-0.046482,-0.012576,-0.104477,-0.129288,0.205406,0.511368,0.311001
2,-0.717147,-0.441298,-0.448934,-0.090037,-0.240685,-0.650738,-1.018500,-1.074817,-0.722548,-0.740732,...,0.182184,-1.214154,-1.268103,-0.972382,-0.294572,-0.506549,-0.467198,-0.293886,-0.403926,-0.429894
3,-0.717147,-0.441298,-0.448934,-0.090037,-0.240685,-2.517052,-2.203617,-1.977765,-1.665060,-0.740732,...,0.212356,-1.462663,-0.728760,0.168090,-1.826885,-1.013849,-0.364022,-0.707328,-0.602243,-0.323230
4,-0.717147,-0.441298,-0.448934,-0.090037,-0.240685,-1.345454,-1.718255,-1.826433,-0.206577,-0.916960,...,0.210568,-0.932118,-1.029693,-0.227616,-1.690103,-1.259500,-0.437955,-0.873988,-0.826831,-0.366760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,-0.974551,-0.799138,-0.863822,-0.641847,-0.852618,-1.760517,-2.460569,-2.559553,-2.591184,-2.092855,...,-0.654884,-0.962030,-1.360201,-1.104305,0.532859,-0.042428,-0.182130,0.614586,-0.345288,-0.707457
4388,-0.974551,-0.799138,-0.863822,-0.641847,-0.852618,-2.293329,-2.006567,-1.693597,-2.571606,-2.092855,...,-0.730916,-0.038738,-0.352739,-0.513432,0.231915,-0.428565,-0.717638,0.472284,-0.304180,-0.637677
4389,-0.974551,-0.799138,-0.863822,-0.641847,-0.852618,-3.202937,-2.671123,-2.258570,-2.843665,-2.092855,...,-0.052572,0.192142,0.499114,0.791479,-0.394813,-0.466705,-0.310697,0.328165,-0.021803,-0.068632
4390,-0.974551,-0.799138,-0.863822,-0.641847,-0.852618,-2.759908,-2.584598,-2.312377,-2.692145,-2.657410,...,0.081884,2.270494,2.438514,1.648877,1.182555,1.121497,0.785503,0.952749,0.582346,0.222246


And apply the 6 models to the application dataset... but first we'll need to grab the mean/standard deviation values for back calculating the temperature from standardized values. 

In [31]:
nn_dir = "~/OneDrive - Colostate/NASA-Northern/data/NN_train_val_test/"
fn = os.path.join(nn_dir, "mean_std_training_v2023-11-08.csv")
mean_std_df = pd.read_csv(fn)
mean_std_df = mean_std_df.set_index("Unnamed: 0")
mean_t = mean_std_df.loc["value"]["mean"]
std_t = mean_std_df.loc["value"]["std"]

model_1_pred = predict_values_test(model_1, standardized_df, mean_t, std_t)
model_2_pred = predict_values_test(model_2, standardized_df, mean_t, std_t)
model_3_pred = predict_values_test(model_3, standardized_df, mean_t, std_t)
model_4_pred = predict_values_test(model_4, standardized_df, mean_t, std_t)
model_5_pred = predict_values_test(model_5, standardized_df, mean_t, std_t)
model_6_pred = predict_values_test(model_6, standardized_df, mean_t, std_t)


 83/138 [=================>............] - ETA: 0s

2023-12-12 21:54:46.470682: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 70/138 [==============>...............] - ETA: 0s

2023-12-12 21:54:46.845615: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 75/138 [===============>..............] - ETA: 0s

...

 85/138 [=================>............] - ETA: 0s

2023-12-12 21:54:47.614813: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 82/138 [================>.............] - ETA: 0s

2023-12-12 21:54:47.968608: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


138/138 [==============================] - 0s 2ms/step


In [37]:

# Create DataFrames for each model's predictions
df_model_2 = pd.DataFrame(model_2_pred, columns=['model_2'])
df_model_3 = pd.DataFrame(model_3_pred, columns=['model_3'])
df_model_4 = pd.DataFrame(model_4_pred, columns=['model_4'])
df_model_5 = pd.DataFrame(model_5_pred, columns=['model_5'])
df_model_6 = pd.DataFrame(model_6_pred, columns=['model_6'])

# Concatenate the DataFrames
df_pred = pd.concat([df_pred, df_model_2, df_model_3, df_model_4, df_model_5, df_model_6], axis=1)

df_pred

,model_1,model_2,model_3,model_4,model_5,model_6
0,7.811100,8.383332,7.734830,7.318177,9.036152,8.360751
1,8.816969,9.192480,8.013252,7.352659,10.511101,9.022324
2,8.235538,8.097383,6.961335,6.289062,9.514428,7.938637
3,3.715522,3.354447,4.012117,3.518452,4.586630,4.232390
4,6.128467,5.779923,5.551198,4.951323,6.819191,6.612491
...,...,...,...,...,...,...
4387,5.352077,4.832039,3.526244,1.606226,2.882202,5.426313
4388,5.800851,4.879270,3.934685,2.180496,3.082332,5.686102
4389,4.060468,3.399368,2.872798,0.511897,0.563210,4.689826
4390,4.718650,4.227302,3.400045,1.250182,1.594637,5.218611


And then save the file

In [38]:
df_pred.to_csv(os.path.join(app_dir, "model_predictions_2020-2022_apr-nov.csv"))